In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

# Dublin的地理位置
dublin_lat = 53.35
dublin_lon = -6.26

# 使用Open-Meteo API获取Dublin当前天气和未来7天逐小时预测
url = f"https://api.open-meteo.com/v1/forecast?latitude={dublin_lat}&longitude={dublin_lon}&current=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,weather_code,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m&timezone=Europe/Dublin&forecast_days=14"

response = requests.get(url)
weather_data = response.json()

print("=" * 80)
print("Dublin当前天气信息:")
print("=" * 80)
if 'current' in weather_data:
    current = weather_data['current']
    print(f"\n当前时间: {current.get('time')}")
    print(f"温度: {current.get('temperature_2m')}°C")
    print(f"体感温度: {current.get('apparent_temperature')}°C")
    print(f"湿度: {current.get('relative_humidity_2m')}%")
    print(f"降水量: {current.get('precipitation')}mm")
    print(f"风速: {current.get('wind_speed_10m')}km/h")

# 处理未来7天小时预报数据
print("\n" + "=" * 80)
print("未来14天逐小时天气预报分析")
print("=" * 80)

if 'hourly' in weather_data:
    hourly = weather_data['hourly']
    times = hourly['time']
    temps = hourly['temperature_2m']
    humidity = hourly['relative_humidity_2m']
    precipitation = hourly['precipitation']
    wind_speed = hourly['wind_speed_10m']
    
    # 创建DataFrame用于分析
    forecast_df = pd.DataFrame({
        '时间': times,
        '温度(°C)': temps,
        '湿度(%)': humidity,
        '降水(mm)': precipitation,
        '风速(km/h)': wind_speed
    })
    
    # 找出风力最大的时间
    max_wind_idx = forecast_df['风速(km/h)'].idxmax()
    max_wind_time = forecast_df.loc[max_wind_idx, '时间']
    max_wind_speed = forecast_df.loc[max_wind_idx, '风速(km/h)']
    
    # 计算距离现在的小时数
    current_time = datetime.fromisoformat(current.get('time'))
    max_wind_datetime = datetime.fromisoformat(max_wind_time)
    hours_until_max_wind = int((max_wind_datetime - current_time).total_seconds() / 3600)
    
    print(f"\n【风力最强时刻】")
    print(f"时间: {max_wind_time}")
    print(f"风速: {max_wind_speed} km/h")
    print(f"距离现在: {hours_until_max_wind} 小时")
    
    # 显示风力最强时刻的完整信息
    print(f"\n【该时刻的详细信息】")
    print(f"温度: {forecast_df.loc[max_wind_idx, '温度(°C)']}°C")
    print(f"湿度: {forecast_df.loc[max_wind_idx, '湿度(%)']}%")
    print(f"降水: {forecast_df.loc[max_wind_idx, '降水(mm)']}mm")
    
    # 额外统计信息
    print("\n" + "=" * 80)
    print("14天风力统计摘要")
    print("=" * 80)
    print(f"最大风速: {max(wind_speed):.1f}km/h")
    print(f"最小风速: {min(wind_speed):.1f}km/h")
    print(f"平均风速: {sum(wind_speed)/len(wind_speed):.1f}km/h")
    
    # 显示Top 5风力最强的时间
    print("\n【未来14天风力Top 5】")
    top_5_wind = forecast_df.nlargest(5, '风速(km/h)')[['时间', '风速(km/h)']]
    for idx, (time, speed) in enumerate(zip(top_5_wind['时间'], top_5_wind['风速(km/h)']), 1):
        wind_datetime = datetime.fromisoformat(time)
        hours_diff = int((wind_datetime - current_time).total_seconds() / 3600)
        print(f"{idx}. {time} - {speed:.1f} km/h (距离现在 {hours_diff} 小时)")

Dublin当前天气信息:

当前时间: 2026-02-03T10:30
温度: 6.1°C
体感温度: 0.3°C
湿度: 87%
降水量: 0.5mm
风速: 30.2km/h

未来14天逐小时天气预报分析

【风力最强时刻】
时间: 2026-02-05T06:00
风速: 43.9 km/h
距离现在: 43 小时

【该时刻的详细信息】
温度: 7.3°C
湿度: 87%
降水: 0.1mm

14天风力统计摘要
最大风速: 43.9km/h
最小风速: 1.2km/h
平均风速: 19.6km/h

【未来14天风力Top 5】
1. 2026-02-05T06:00 - 43.9 km/h (距离现在 43 小时)
2. 2026-02-05T08:00 - 43.6 km/h (距离现在 45 小时)
3. 2026-02-05T07:00 - 42.5 km/h (距离现在 44 小时)
4. 2026-02-06T05:00 - 41.8 km/h (距离现在 66 小时)
5. 2026-02-06T04:00 - 40.8 km/h (距离现在 65 小时)


In [2]:
import requests

all_articles = []
api_key = "37d41edc-7d44-4947-a1bf-e23550603052"

for page in range(1, 3):
    # 添加 show-fields=body 获取文章正文
    url = f"https://content.guardianapis.com/search?q=ireland&page-size=50&page={page}&show-fields=body,trailText&api-key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        results = data['response']['results']
        all_articles.extend(results)
        print(f"已获取第 {page} 页，共 {len(results)} 篇文章。")
    else:
        print(f"请求第 {page} 页时出错，状态码: {response.status_code}")

print(f"\n总共获取了 {len(all_articles)} 篇文章。\n")
for idx, article in enumerate(all_articles, 1):
    title = article['webTitle']
    # 优先获取bodyText，如果没有就用trailText
    body = article.get('fields', {}).get('bodyText', article.get('fields', {}).get('trailText', ''))[:100]
    publish_date = article.get('webPublicationDate', '未知日期')[:10]
    section = article.get('sectionName', '未分类')
    print(f"{idx}. {title}")
    print(f"发布日期: {publish_date} | 分类: {section}") 
    print(f"摘要:{body}")
    print("-" * 80)

已获取第 1 页，共 50 篇文章。
已获取第 2 页，共 50 篇文章。

总共获取了 100 篇文章。

1. I ran 1,400 miles around Ireland
发布日期: 2026-01-05 | 分类: Travel
摘要:On a running pilgrimage in the land of my forebears I was blown away by the scenery – and even more 
--------------------------------------------------------------------------------
2. World Cup playoff draw: Wales face Bosnia, Northern Ireland head to Italy, Ireland go to Prague
发布日期: 2025-11-20 | 分类: Football
摘要:Wales and Northern Ireland will do battle for a place at the 2026 World Cup if they navigate playoff
--------------------------------------------------------------------------------
3. ‘Target mainland’: planned Troubles board game condemned in Northern Ireland
发布日期: 2026-01-23 | 分类: UK news
摘要:By turning conflict into entertainment US games company is ignoring its living legacy, says victims 
--------------------------------------------------------------------------------
4. World Cup 2026 playoffs draw: Wales v Bosnia-Herzegovina, Czech Republic v Repu

In [5]:
import requests
import folium
import pandas as pd

# 1. 获取实时数据 (JCDecaux)
api_key = "11c0b077ab18a5d4f761dc7b7469d89b7f5e22b3"
status_url = f"https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey={api_key}"

# 获取站点信息与实时状态 (同一接口返回)
stations = requests.get(status_url).json()

# 转为 DataFrame 方便处理
df = pd.DataFrame(stations)

# 2. 初始化地图 (以都柏林市中心为中心)
m = folium.Map(location=[53.3498, -6.2603], zoom_start=14, tiles='cartodbpositron')

# 3. 将站点添加到地图
for _, row in df.iterrows():
    # 定义颜色逻辑：没车红色，车少橙色，车多绿色
    available = row["available_bikes"]
    color = "green" if available > 5 else "orange" if available > 0 else "red"
    pos = row["position"]
    
    folium.CircleMarker(
        location=[pos["lat"], pos["lng"]],
        radius=7,
        popup=f"<b>{row['name']}</b><br>Available: {available}/{row['bike_stands']}",
        color=color,
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 4. 在 Jupyter 中直接显示地图
m


In [4]:
import json
import time
from datetime import datetime, timezone, timedelta
from pathlib import Path

# 每5分钟抓取一次，保存为独立 JSON 文件；出错不影响下一轮
api_key = "11c0b077ab18a5d4f761dc7b7469d89b7f5e22b3"
status_url = f"https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey={api_key}"
output_dir = Path("data/dublinbike_status")
output_dir.mkdir(parents=True, exist_ok=True)

def fetch_and_save_once():
    ts = datetime.now(timezone.utc)
    ts_str = ts.strftime("%Y%m%dT%H%M%SZ")
    filename = output_dir / f"station_status_{ts_str}.json"
    try:
        resp = requests.get(status_url, timeout=20)
        resp.raise_for_status()
        data = resp.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)
        print(f"saved: {filename}")
    except Exception as e:
        print(f"error: {e}; will retry next cycle")

# 运行48小时自动停止
max_hours = 48
end_time = datetime.now(timezone.utc) + timedelta(hours=max_hours)

# 运行时可按下停止按钮中断
while datetime.now(timezone.utc) < end_time:
    fetch_and_save_once()
    time.sleep(300)
print("done: reached 48h timeout")


saved: data/dublinbike_status/station_status_20260203T110548Z.json


KeyboardInterrupt: 